In [216]:
import pandas as pd
from proto_tools.codec.tracking import TrackedPeople, TrackedPerson
import logging
import argparse
import typing
logging.getLogger("py4j").setLevel(logging.ERROR)
import json
import re
from proto_tools.codec.tracking import TrackedPeople, TrackedPerson
import os
import pickle as pk
import bson
import numpy as np
from google.protobuf.json_format import MessageToDict
from typing import List


from pandas import DataFrame
from dataclasses import dataclass
import os
import requests
import json
import re
from record_dataloader import build_loader
from proto_tools.codec.tracking import TrackedPeople
from tqdm import tqdm
import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import json
import math
import numpy as np
def calc_speed(person_x,person_y):
    abs_time=abs(person_x.timestamp-person_y.timestamp)
    dists=[]
    dist=0
    dists.append(person_x.head.motion.position.x-person_y.head.motion.position.x)
    dists.append(person_x.head.motion.position.y-person_y.head.motion.position.y)
    dists.append(person_x.head.motion.position.z-person_y.head.motion.position.z)
    for i in dists:
        dist=dist+i**2
    if dist>0 and abs_time==0:
        return 9999999
    if dist==0 and abs_time==0:
        return 0
    #   if abs_time != 0:
    #     print("-----------------------",math.sqrt(dist)/abs_time)
    return math.sqrt(dist)/abs_time
def search_closest(ts,timestamp):
    index = np.searchsorted(ts, timestamp)
    if index >= len(ts):
        index = len(ts) - 1
    elif index > 0 and (timestamp - ts[index - 1]) < (ts[index] - timestamp):
        index = index - 1
    if index < 0:
        print("error ,need be check !")
        exit()
    delta = np.abs(ts[index] - timestamp)
    return index
def check_trips(persons_x,persons_y,sample_num=50):
    ts_min=max(persons_x[0].timestamp,persons_y[0].timestamp)
    ts_max=min(persons_x[-1].timestamp,persons_y[-1].timestamp)
    #     no intersect
#     print(ts_min,ts_max)
    if ts_min>=ts_max:
        if persons_x[0].timestamp<persons_y[0].timestamp:
            return calc_speed(persons_x[-1],persons_y[0]),calc_speed(persons_x[-1],persons_y[0])
        else:
            return calc_speed(persons_x[0],persons_y[-1]),calc_speed(persons_x[0],persons_y[-1])

    ts_x=[person.timestamp for person in persons_x]
    ts_y=[person.timestamp for person in persons_y]
    #   print(len(ts_x),len(ts_y))
    MAX=0
    MIN=9999999
    for ts in np.linspace(ts_min,ts_max,50,endpoint=True):
        index_x=search_closest(ts_x,ts)
        index_y=search_closest(ts_y,ts)
        dist=calc_speed(persons_x[index_x],persons_y[index_y])
        MAX=max(MAX,dist)
        MIN=min(MIN,dist)
#         print(dist,MIN,MAX)
    #     print(MAX,MIN)
    #     print(MIN,MAX)
    return MIN,MAX
def get_dict_in_rec(data_dir:str,file_name:str="tracked_people.recp"):
    target_loader = build_loader(
        data_dir,
        file_name
    )
    peoples={}
    for anno_pb, ts in tqdm(
            target_loader.get_iter(out_ts=True), desc="load target", total=len(target_loader)
    ):
        tracked_people=TrackedPeople.from_pb(anno_pb)
        for person in tracked_people.people:
            target_id = str(person.tracking_id)
            person.timestamp=tracked_people.timestamp
            #     print(peoples.__contains__(target_id))
            if not peoples.__contains__(target_id):
                peoples[target_id]=[]
            #       print(peoples)
            peoples[target_id].append([ts,person])
    return peoples
def get_person_list(id,df,peoples):
    df_tmp=df[df["corrected_shopper_id"]==id]
    list_tmp=df_tmp["shopper_id"].tolist()
    list_tmp.append(id)
    list_tmp=list(set(list_tmp))
    persons=[]
    timestamps=[]
    for id in list_tmp:
        if not peoples.__contains__(str(id)):
            continue
        for ts,person in peoples[str(id)]:
            timestamps.append(ts)
            persons.append(person)
    argsort = np.argsort(timestamps)
    persons = [persons[i] for i in argsort]
    return persons

In [99]:
path="~/Downloads/merged_new_hat_table.csv"
df=pd.read_csv(path)

In [100]:
df

,Unnamed: 0,Unnamed: 0.1,order_id,session_id,customer_id,shopper_id,corrected_shopper_id,t_start,t_end,source,...,quantity,event_type,start_ts_x,end_ts_x,record,case_id,id,videos,start_ts_y,end_ts_y
0,0,39299,44710,fba428cf-ebde-4ff4-b594-6c251579f883,6622,11510217257579097954,11510217257579097954,2021-11-23 12:24:34.000,2021-11-23 12:24:40.000,HAT,...,1,shopping,1637670274,1637670280,rec-1637670153-1637670753,3e661d69-c930-458d-946a-8647b0366bd7,3e661d69-c930-458d-946a-8647b0366bd7,[Row(id='e3d1e9dd-de97-497d-be1a-b6c9e2de37c9'...,2021-11-23 12:24:34.327,2021-11-23 12:24:40.284
1,1,39304,44710,fba428cf-ebde-4ff4-b594-6c251579f883,6622,11510217257579097954,11510217257579097954,2021-11-23 12:24:47.000,2021-11-23 12:24:54.000,HAT,...,1,shopping,1637670287,1637670294,rec-1637670153-1637670753,c669d5ca-0977-4a1e-a018-219cdfe95523,c669d5ca-0977-4a1e-a018-219cdfe95523,[Row(id='6e2dba62-0f4e-4fdf-8ef3-786a2220fa2e'...,2021-11-23 12:24:47.325,2021-11-23 12:24:54.155
2,2,39306,44651,d9be0234-686c-4d32-8272-9338407a056f,6624,13611814041990991150,13611814041990991150,2021-11-23 12:24:47.000,2021-11-23 12:24:53.000,HAT,...,1,shopping,1637670287,1637670293,rec-1637670153-1637670753,a0ba4ce6-66f2-4164-b28d-63b54e503bee,a0ba4ce6-66f2-4164-b28d-63b54e503bee,[Row(id='88c81370-c510-4eb7-87f7-63d3a7c6a968'...,2021-11-23 12:24:47.727,2021-11-23 12:24:53.739
3,3,39308,44682,e42840ab-562e-424c-b065-db81bbc012da,6626,8687438450198398278,8687438450198398278,2021-11-23 12:24:48.000,2021-11-23 12:24:58.000,HAT,...,1,shopping,1637670288,1637670298,rec-1637670153-1637670753,d5a58f5e-72a2-4945-a6ec-cac0faa7b16e,d5a58f5e-72a2-4945-a6ec-cac0faa7b16e,[Row(id='8bb95770-48bb-4b3b-b494-3f9e251ff540'...,2021-11-23 12:24:48.978,2021-11-23 12:24:58.262
4,4,39322,44651,d9be0234-686c-4d32-8272-9338407a056f,6624,13611814041990991150,13611814041990991150,2021-11-23 12:25:07.000,2021-11-23 12:25:28.000,HAT,...,1,shopping,1637670307,1637670328,rec-1637670153-1637670753,3635e2de-e818-4c88-b1d2-47a3ccbacfb2,3635e2de-e818-4c88-b1d2-47a3ccbacfb2,[Row(id='58929894-c4ca-4381-a7ac-a04462684ff6'...,2021-11-23 12:25:07.614,2021-11-23 12:25:28.127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4482,4482,60703,44849,d1128d9a-5c7d-4474-8991-60a1758cc322,7088,6729549988696602420,6729549988696602420,2021-11-24 12:12:50.000,2021-11-24 12:12:56.000,HAT,...,-1,shopping,1637755970,1637755976,rec-1637755641-1637756241,64ed112d-4316-47a1-ad16-801e71c704ee,64ed112d-4316-47a1-ad16-801e71c704ee,[Row(id='504187ba-468b-4ea1-acb4-0e5064925057'...,2021-11-24 12:12:50.176,2021-11-24 12:12:56.406
4483,4483,60704,44849,d1128d9a-5c7d-4474-8991-60a1758cc322,7088,6729549988696602420,6729549988696602420,2021-11-24 12:12:50.000,2021-11-24 12:12:56.000,HAT,...,0,shopping,1637755970,1637755976,rec-1637755641-1637756241,64ed112d-4316-47a1-ad16-801e71c704ee,64ed112d-4316-47a1-ad16-801e71c704ee,[Row(id='504187ba-468b-4ea1-acb4-0e5064925057'...,2021-11-24 12:12:50.176,2021-11-24 12:12:56.406
4484,4484,60705,44849,d1128d9a-5c7d-4474-8991-60a1758cc322,7088,6729549988696602420,6729549988696602420,2021-11-24 12:12:50.000,2021-11-24 12:12:56.000,HAT,...,0,shopping,1637755970,1637755976,rec-1637755641-1637756241,64ed112d-4316-47a1-ad16-801e71c704ee,64ed112d-4316-47a1-ad16-801e71c704ee,[Row(id='504187ba-468b-4ea1-acb4-0e5064925057'...,2021-11-24 12:12:50.176,2021-11-24 12:12:56.406
4485,4485,60706,44849,d1128d9a-5c7d-4474-8991-60a1758cc322,7088,6729549988696602420,6729549988696602420,2021-11-24 12:12:50.000,2021-11-24 12:12:56.000,HAT,...,0,shopping,1637755970,1637755976,rec-1637755641-1637756241,64ed112d-4316-47a1-ad16-801e71c704ee,64ed112d-4316-47a1-ad16-801e71c704ee,[Row(id='504187ba-468b-4ea1-acb4-0e5064925057'...,2021-11-24 12:12:50.176,2021-11-24 12:12:56.406


In [101]:
ids=df["corrected_shopper_id"].tolist()
ids=list(set(ids))
print(len(ids))
recs=df["record"].tolist()
recs=list(set(recs))

202


In [102]:
path="/Users/julyc/Downloads"

In [103]:
rec_name='rec-1637753840-1637754440'

In [89]:
peoples=get_dict_in_rec(path,file_name="tracked_people.recp")

INFO:record_dataloader.loaders.time_series_loader:Load timestamp cache from /Users/julyc/Downloads/timestamp_cache.pk
INFO:record_dataloader.loaders.bson_loader:Set tracked_people range [-inf, inf]
Load tracked_people: 100%|███████████████████| 8998/8998 [00:15<00:00, 575.39it/s]
INFO:record_dataloader.loaders.bson_loader:Load 8998 `tracked_people.recp` from /Users/julyc/Downloads
load target: 100%|████████████████████████████| 8998/8998 [06:09<00:00, 24.38it/s]


In [145]:
for id,value in peoples.items():
    print(id,value[0][0],value[-1][0])

800699710946761559 1637753240.5117621 1637754440.6430619
644721220759728265 1637753240.5117621 1637754440.6430619
15941934107840633359 1637753240.5117621 1637754440.6430619
1050904838066235841 1637753240.5117621 1637753417.5638924
5061917154731132964 1637753240.5117621 1637753443.2210596
13903044462240620871 1637753240.5117621 1637753441.2932837
17140032195424231850 1637753240.5117621 1637753393.0194004
4202631835035341525 1637753240.5117621 1637753605.6843534
18212758567991666566 1637753240.5117621 1637754440.6430619
1354849430209566791 1637753240.5117621 1637753664.5178618
2147318807454420666 1637753240.5117621 1637753560.1644301
10393503135692674691 1637753240.5117621 1637753553.749516
9914065956746329164 1637753240.5117621 1637753541.888859
5170782221654280024 1637753240.5117621 1637753632.459812
1500310280052940564 1637753240.5117621 1637753284.8681636
14127829359021775992 1637753240.5117621 1637753513.8779137
1517887004371362023 1637753240.5117621 1637753452.3553278
1100961653762

In [104]:
peoples_ids=list(peoples.keys())

In [158]:
df_tmp=df[df["record"]==rec_name]
ids=df_tmp["corrected_shopper_id"].tolist()
ids=list(set(ids))
# print(len(ids))

73


In [169]:
rec_name

'rec-1637753840-1637754440'

In [170]:
df_tmp=df[df["record"]==rec_name]
df_tmp

,Unnamed: 0,Unnamed: 0.1,order_id,session_id,customer_id,shopper_id,corrected_shopper_id,t_start,t_end,source,...,quantity,event_type,start_ts_x,end_ts_x,record,case_id,id,videos,start_ts_y,end_ts_y
2606,2606,50280,44955,d5a3b892-c703-4963-b55b-f1e5ab33cd26,224,11358782563247001443,14386716065400302779,2021-11-24 11:37:59.000,2021-11-24 11:38:06.000,HAT,...,1,shopping,1637753879,1637753886,rec-1637753840-1637754440,fb8fac1f-8089-40c0-bbbd-daa4e3bd6cbe,fb8fac1f-8089-40c0-bbbd-daa4e3bd6cbe,[Row(id='5d9abf14-22ca-4e68-b015-707c3e660924'...,2021-11-24 11:38:03.995,2021-11-24 11:38:08.579
2607,2607,50356,44955,d5a3b892-c703-4963-b55b-f1e5ab33cd26,224,11358782563247001443,14386716065400302779,2021-11-24 11:38:15.000,2021-11-24 11:38:21.000,HAT,...,1,shopping,1637753895,1637753901,rec-1637753840-1637754440,fb8fac1f-8089-40c0-bbbd-daa4e3bd6cbe,fb8fac1f-8089-40c0-bbbd-daa4e3bd6cbe,[Row(id='5d9abf14-22ca-4e68-b015-707c3e660924'...,2021-11-24 11:38:03.995,2021-11-24 11:38:08.579
2616,2616,50127,44823,caa52be8-54e4-41a8-9d67-41fa08772cfa,7023,11228063317183815026,11228063317183815026,2021-11-24 11:37:15.000,2021-11-24 11:37:36.000,HAT,...,1,shopping,1637753835,1637753856,rec-1637753840-1637754440,66a4e8fc-b516-4828-85b4-a21f46fa52f1,66a4e8fc-b516-4828-85b4-a21f46fa52f1,[Row(id='1f25d58c-e986-4f54-a78e-0de2dbbcff24'...,2021-11-24 11:37:15.015,2021-11-24 11:37:36.127
2617,2617,50139,44919,6870e457-1635-46ee-a62b-106a4ee04ea1,7015,12618980535252031476,12618980535252031476,2021-11-24 11:37:19.000,2021-11-24 11:37:24.000,HAT,...,1,shopping,1637753839,1637753844,rec-1637753840-1637754440,2d14db3c-2b62-4668-a7a4-4161363e0cfe,2d14db3c-2b62-4668-a7a4-4161363e0cfe,[Row(id='c531dab2-5f3d-4179-a3ba-98f6cd84a405'...,2021-11-24 11:37:19.347,2021-11-24 11:37:24.410
2618,2618,50140,44919,6870e457-1635-46ee-a62b-106a4ee04ea1,7015,12618980535252031476,12618980535252031476,2021-11-24 11:37:19.000,2021-11-24 11:37:24.000,HAT,...,1,shopping,1637753839,1637753844,rec-1637753840-1637754440,2d14db3c-2b62-4668-a7a4-4161363e0cfe,2d14db3c-2b62-4668-a7a4-4161363e0cfe,[Row(id='c531dab2-5f3d-4179-a3ba-98f6cd84a405'...,2021-11-24 11:37:19.347,2021-11-24 11:37:24.410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3286,3286,53570,44900,7fcd3f15-611b-4740-b538-61707d0004f8,7073,3380476243277985776,3380476243277985776,2021-11-24 11:47:11.000,2021-11-24 11:47:15.000,HAT,...,0,shopping,1637754431,1637754435,rec-1637753840-1637754440,5586b65e-fecf-49d3-879c-af7e0eb23c45,5586b65e-fecf-49d3-879c-af7e0eb23c45,[Row(id='08d68c6d-795d-4b68-a998-7794fea7ee71'...,2021-11-24 11:47:11.210,2021-11-24 11:47:15.913
3287,3287,53574,44945,8d57ed1e-2fd1-4ae6-b105-130acd34e60c,7075,11997167441761800692,11997167441761800692,2021-11-24 11:47:11.000,2021-11-24 11:47:17.000,HAT,...,1,shopping,1637754431,1637754437,rec-1637753840-1637754440,cdf3f06c-5a12-437b-95f3-528968d4911d,cdf3f06c-5a12-437b-95f3-528968d4911d,[Row(id='46f6ed71-c40c-4085-877b-ecb3beb53dbf'...,2021-11-24 11:47:11.913,2021-11-24 11:47:17.372
3288,3288,53575,44945,8d57ed1e-2fd1-4ae6-b105-130acd34e60c,7075,11997167441761800692,11997167441761800692,2021-11-24 11:47:11.000,2021-11-24 11:47:17.000,HAT,...,1,shopping,1637754431,1637754437,rec-1637753840-1637754440,cdf3f06c-5a12-437b-95f3-528968d4911d,cdf3f06c-5a12-437b-95f3-528968d4911d,[Row(id='46f6ed71-c40c-4085-877b-ecb3beb53dbf'...,2021-11-24 11:47:11.913,2021-11-24 11:47:17.372
3289,3289,53581,44844,9806dad9-a056-44b9-8f19-7b43d437c568,7059,8955009961582941638,8955009961582941638,2021-11-24 11:47:13.000,2021-11-24 11:47:18.000,HAT,...,1,shopping,1637754433,1637754438,rec-1637753840-1637754440,da09870c-8ebd-47d9-817a-3fa89b5b4f16,da09870c-8ebd-47d9-817a-3fa89b5b4f16,[Row(id='883490ee-a09f-4934-9232-ac0a453ace14'...,2021-11-24 11:47:13.099,2021-11-24 11:47:18.739


In [ ]:
valid_pairs=[]
for rec in recs:
    peoples=get_dict_in_rec(os.path.join(path,rec),file_name="tracked_people.recp")
    df_tmp=df[df["record"]==rec]
    ids=df_tmp["corrected_shopper_id"].tolist()
    ids=list(set(ids))
    for i,id1 in tqdm(enumerate(ids),total=len(ids)):
        for j,id2 in enumerate(ids):
            if i>=j:
                continue
            id1_list=get_person_list(id1,df,peoples)
            id2_list=get_person_list(id2,df,peoples)
            MIN,MAX=check_trips(id1_list,id2_list)
#             if MAX != 9999999:
#                 print(id1,id2,MIN,MAX)
            if MAX >=10:
                continue
            valid_pairs.append((id1,id2))


In [219]:
se=pd.Series(valid_pairs)

In [221]:
se.to_csv(os.path.join(path,"se.csv"))